In [11]:
import transformers
import pandas as pd
from datasets import load_dataset

from app.model import IntentClassifier
from app.atis.utils import ATIS_INTENT_MAPPING as intent_mapping

dataset = load_dataset("tuetschek/atis")

In [2]:
from transformers import pipeline
# Bart-Large-MNLI 180M parameters
classifier = pipeline("zero-shot-classification",
                      model="google/flan-t5-base", device="cuda")


Some weights of T5ForSequenceClassification were not initialized from the model checkpoint at google/flan-t5-base and are newly initialized: ['classification_head.dense.bias', 'classification_head.dense.weight', 'classification_head.out_proj.bias', 'classification_head.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Failed to determine 'entailment' label id from the label2id mapping in the model config. Setting to -1. Define a descriptive label2id mapping in the model config to ensure correct outputs.


In [6]:
# use intent mapping names
candidate_labels = list(intent_mapping.values())
classifier(dataset["train"][0]["text"], candidate_labels)

{'sequence': 'i want to fly from boston at 838 am and arrive in denver at 1110 in the morning',
 'labels': ['Ground Transportation Cost Inquiry',
  'Aircraft Type Inquiry',
  'Ground Transportation Inquiry',
  'Airport Location Inquiry',
  'Flight Restriction Inquiry',
  'Flight Quantity Inquiry',
  'Airport Distance Inquiry',
  'Aircraft Seating Capacity Inquiry',
  'Flight Schedule Inquiry',
  'Cheapest Fare Inquiry',
  'Flight Number Inquiry',
  'Airport Ground Transportation and Cost Query',
  'Airfare Information Requests',
  'Airport Information and Queries',
  'Abbreviation and Fare Code Meaning Inquiry',
  'Inquiry about In-flight Meals',
  'Flight Booking Request',
  'Airline Information Request'],
 'scores': [0.05936965346336365,
  0.058922287076711655,
  0.05849437043070793,
  0.05739816650748253,
  0.057348936796188354,
  0.05706634372472763,
  0.056799400597810745,
  0.05678324028849602,
  0.056043293327093124,
  0.055784646421670914,
  0.05563261732459068,
  0.05504035949

In [9]:
from tqdm import tqdm
results = []
for row in tqdm(dataset["test"]):
    intent = row["intent"] 
    if intent not in intent_mapping:
        continue 
    
    prediction = classifier(row["text"], candidate_labels)
    results.append({"prediction": prediction, "y": intent_mapping[intent]})

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 893/893 [02:25<00:00,  6.12it/s]


In [12]:
from sklearn.metrics import classification_report
y = [r["y"] for r in results]
predictions = [r["prediction"] for r in results]

prediction_labels = []
for row in predictions:
    max_index = row['scores'].index(max(row['scores']))
    label = row['labels'][max_index]
    prediction_labels.append(label)
print(len(prediction_labels))

pd.DataFrame(classification_report(y, prediction_labels, output_dict=True)).T

876


/home/serj/dev/customer_support_classifier/venv/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/home/serj/dev/customer_support_classifier/venv/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1509: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/home/serj/dev/customer_support_classifier/venv/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average

,precision,recall,f1-score,support
Abbreviation and Fare Code Meaning Inquiry,0.111111,0.090909,0.100000,33.000000
Aircraft Seating Capacity Inquiry,0.000000,0.000000,0.000000,21.000000
Aircraft Type Inquiry,0.007587,0.555556,0.014970,9.000000
Airfare Information Requests,0.333333,0.020833,0.039216,48.000000
Airline Information Request,0.000000,0.000000,0.000000,38.000000
Airport Distance Inquiry,0.000000,0.000000,0.000000,10.000000
Airport Ground Transportation and Cost Query,0.000000,0.000000,0.000000,0.000000
Airport Information and Queries,0.000000,0.000000,0.000000,18.000000
Airport Location Inquiry,0.000000,0.000000,0.000000,6.000000
Cheapest Fare Inquiry,0.000000,0.000000,0.000000,0.000000
